<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">M3.4</span> – إنشاء ميزانية مائية على مستوى الحوض</h1><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">جزء من: <a href="https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources" dir="ltr">Open Science for Water Resources</a></p>
</div>

In [ ]:
import glob
import datetime
import earthaccess
import numpy as np
import h5py
import xarray as xr
import rasterio as rio
from matplotlib import pyplot

auth = earthaccess.login()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">هدفنا حساب ميزانية مياه لحوض نهر <span dir="ltr" style="unicode-bidi: isolate;">Yellowstone</span>. حتى الآن حسبنا الهطول <span dir="ltr" style="unicode-bidi: isolate;">P</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">$$
P = E + R + \Delta S
$$</span></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في هذا الدرس سنضيف تقدير النتح–التبخر <span dir="ltr" style="unicode-bidi: isolate;">ET</span>، ثم الجريان/التصريف <span dir="ltr" style="unicode-bidi: isolate;">R</span>، وأخيرًا سنحسب التغير في التخزين <span dir="ltr" style="unicode-bidi: isolate;">ΔS</span>.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تقدير النتح–التبخر للحوض</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سنستخدم منتج <a href="https://dx.doi.org/10.5067/MODIS/MOD16A3GF.061" dir="ltr">NASA MODIS MOD16</a> لتقدير <span dir="ltr" style="unicode-bidi: isolate;">ET</span> على مستوى الحوض.</p>
</div>

In [ ]:
import geopandas

basin = geopandas.read_file('data/shp/YellowstoneRiver_drainage_WSG84.shp')

In [ ]:
# Provided in (min_X, min_Y, max_X, max_Y) order
basin.bounds.to_numpy().ravel()

In [ ]:
bbox = basin.bounds.to_numpy().ravel()
bbox

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن نحن جاهزون لتنزيل البلاطات التي تتقاطع مع منطقة الدراسة.</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يتيح الوسيط <span dir="ltr" style="unicode-bidi: isolate;">bounding_box</span> في <span dir="ltr" style="unicode-bidi: isolate;">earthaccess.search_data()</span> تحديد مستطيل مكاني لإجراء البحث.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سنستخدم حدودًا تغطي حوضنا ثم نرشّح النتائج للبلاطات المتقاطعة.</li>
</ul>
</div>

In [ ]:
results = earthaccess.search_data(
    short_name = 'MOD16A3GF',
    temporal = ('2014-01-01', '2023-12-31'),
    bounding_box = tuple(bbox))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في الواقع توجد بلاطتان تتقاطعان مع منطقة الدراسة. وللتبسيط، سنتجاهل الجزء الصغير جدًا الواقع في البلاطة الثانية، وسنركز على البلاطة الرئيسية.</p>
</div>

In [ ]:
results_clean = list(filter(lambda granule: 'h10v04' in granule['meta']['native-id'], results))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد ذلك، لننزّل فقط البلاطات التي نهتم بها.</p><h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🎯 أفضل الممارسات</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أننا أنشأنا مكانًا مخصصًا لبيانات <span dir="ltr" style="unicode-bidi: isolate;">MOD16A3</span>: <span dir="ltr" style="unicode-bidi: isolate;">data/MOD16A3</span>. تنظيم البيانات في مجلدات منفصلة يساعد على إعادة الإنتاج وتجنب الخلط بين المصادر.</p>
</div>

In [ ]:
earthaccess.download(results_clean, 'data/MOD16A3')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تثبيت حزمة Python جديدة</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تأتي بعض منتجات ناسا بصيغ علمية قد تكون صعبة الاستخدام. بيانات <span dir="ltr" style="unicode-bidi: isolate;">MOD16</span> تأتي بصيغة <span dir="ltr" style="unicode-bidi: isolate;">HDF4-EOS</span>؛ لذا سنثبت حزمة تساعدنا على قراءتها بسهولة.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لنفتح ملفًا واحدًا من بيانات <span dir="ltr" style="unicode-bidi: isolate;">MOD16</span> لنرى كيف تعمل مكتبة <span dir="ltr" style="unicode-bidi: isolate;">py4eos</span>.</p>
</div>

In [ ]:
import py4eos

hdf = py4eos.read_hdf4eos('data/MOD16A3/MOD16A3GF.A2014001.h10v04.061.2022077145846.hdf')
hdf

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا استخراج مجموعة بيانات من ملف <span dir="ltr" style="unicode-bidi: isolate;">HDF4-EOS</span> باستخدام الطريقة <span dir="ltr" style="unicode-bidi: isolate;">get()</span>.</p>
</div>

In [ ]:
et = hdf.get('ET_500m')
et

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما في كثير من البيانات العلمية، القيم تكون «مقيسة» لتقليل حجم الملف. هذا صحيح أيضًا لبيانات <span dir="ltr" style="unicode-bidi: isolate;">MOD16</span>. الوحدات تكون عادة <span dir="ltr" style="unicode-bidi: isolate;">mm/year</span>، لذا نطبّق عامل القياس ونحوّل القيم غير الصالحة إلى <span dir="ltr" style="unicode-bidi: isolate;">NaN</span>.</p>
</div>

In [ ]:
et = hdf.get('ET_500m', scale_and_offset = True)
et

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وبدلًا من مصفوفة <span dir="ltr" style="unicode-bidi: isolate;">NumPy</span> يمكننا طلب إرجاع البيانات ككائن <span dir="ltr" style="unicode-bidi: isolate;">rasterio Dataset</span> لتسهيل الإسقاط والقص.</p>
</div>

In [ ]:
ds_et = hdf.to_rasterio('ET_500m', filename = '', driver = 'MEM', scale_and_offset = True)
ds_et

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تجميع سلسلة زمنية للـ ET داخل الحوض</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نريد قصّ بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> إلى حدود الحوض ثم تلخيصها مكانيًا. أولًا نختار نظام إحداثيات مناسب ثم نُسقط البيانات قبل القص.</p>
</div>

In [ ]:
# Project our basin data to the new CRS
basin_albers = basin.to_crs(epsg = 5070)
basin_albers.geometry[0]

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>نريد حساب إجمالي <span dir="ltr" style="unicode-bidi: isolate;">ET</span> للحوض لكل سنة</strong>، لذا سنحتاج إلى العمل على جميع الملفات السنوية التي نزّلناها.</p>
</div>

In [ ]:
# Get a list of the *.hdf files we downloaded
file_list = glob.glob('data/MOD16A3/*.hdf')
file_list

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قبل القص، سنسقط كل صورة <span dir="ltr" style="unicode-bidi: isolate;">MOD16 ET</span> إلى نظام الإحداثيات الجديد. وبما أن <span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> لا يتعامل مباشرة مع <span dir="ltr" style="unicode-bidi: isolate;">HDF4-EOS</span>، فسنحوّل كل صورة إلى <span dir="ltr" style="unicode-bidi: isolate;">GeoTIFF</span> وسيط عند الحاجة، ثم نقصّها.</p>
</div>

In [ ]:
gtiff_file_list = []

for filename in file_list:
    # Extract the year from the filename
    year = filename.split('.')[1].replace('A', '').replace('001', '')
    
    # Read in the MODIS MOD16 data
    hdf = py4eos.read_hdf4eos(filename)
    
    # Create a rasterio Dataset for this raster; also write out a GeoTIFF file
    output_filename = f'processed/MOD16A3GF_{year}_ET_500m.tiff'
    ds_et = hdf.to_rasterio(
        'ET_500m', filename = output_filename, scale_and_offset = True)
    ds_et.close()
    
    # Save the output GeoTIFF filename
    gtiff_file_list.append(output_filename)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🎯 أفضل الممارسات</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أننا نخزن ملفات <span dir="ltr" style="unicode-bidi: isolate;">GeoTIFF</span> الناتجة في مجلد جديد <span dir="ltr" style="unicode-bidi: isolate;">processed</span>. لا نريد خلط هذه الملفات المعالجة مع البيانات الخام.</p>
</div>

In [ ]:
gtiff_file_list.sort()
gtiff_file_list

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قصّ مجموعة بيانات مكانية إلى منطقة أصغر</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن نحن جاهزون لقص بيانات <span dir="ltr" style="unicode-bidi: isolate;">MOD16 ET</span> إلى حدود الحوض. يمكننا القص باستخدام <span dir="ltr" style="unicode-bidi: isolate;">rasterio</span>، ثم سنحوّل الناتج إلى <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لتسهيل العمل الزمني والتحليل.</p>
</div>

In [ ]:
import rioxarray

ds_et = rioxarray.open_rasterio(gtiff_file_list[0])

# Project the MOD16 data to match the CRS of our basin
ds_et_albers = ds_et.rio.reproject(basin_albers.crs)

# Clip the MOD16 data to the boundary of our basin; plot the result
ds_et_albers.rio.clip(basin_albers.geometry.values).plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">من المفيد في هذا الرسم أننا نستطيع رؤية نهر <span dir="ltr" style="unicode-bidi: isolate;">Yellowstone</span> عند خروجه من الحوض (أعلى يمين الصورة) قرب <span dir="ltr" style="unicode-bidi: isolate;">Sidney, Montana</span>. <strong>الآن يمكننا تكرار ذلك لكل الملفات السنوية</strong> لبناء سلسلة زمنية كاملة.</p>
</div>

In [ ]:
# Create an empty list to store the basin-scale ET
et_series = []

for filename in gtiff_file_list:
    ds_et = rioxarray.open_rasterio(filename)
    
    # Project the MOD16 data to match the CRS of our basin
    ds_et_albers = ds_et.rio.reproject(basin_albers.crs)
    
    # Clip the MOD16 data to the boundary of our basin
    ds_et_basin = ds_et_albers.rio.clip(basin_albers.geometry.values)
    
    # Again, we take the mean because it is equivalent to spatially
    #    distributing the columns of water in each pixel
    et_series.append(float(ds_et_basin.mean().values))

et_series = np.array(et_series)
et_series

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الحصول على بيانات الجريان على مستوى الحوض</h2>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لدينا الآن جزء إضافي من ميزانية المياه على مستوى الحوض، وما زال ينقصنا الجريان/التصريف. في كثير من المناطق يكون الحصول على الجريان صعبًا؛ وغالبًا ما يتوفر فقط للأحواض المغلقة أو حيث توجد محطات قياس.</p>
</div>

In [ ]:
import xarray as xr

ds = xr.open_dataset('data/HYSETS-2023_watershed_YellowstoneRiver.nc')
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بما أننا ننظر إلى فترة عشر سنوات من <span dir="ltr" style="unicode-bidi: isolate;">2014</span> حتى <span dir="ltr" style="unicode-bidi: isolate;">2023</span>، فلنقصّ بيانات التصريف على المدى الزمني نفسه.</p>
</div>

In [ ]:
ds_10years = ds.sel(time = slice('2014-01-01', '2023-12-31'))
ds_10years['discharge'].plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بيانات التصريف تبدو بوحدات متر مكعب في الثانية <span dir="ltr" style="unicode-bidi: isolate;">m³/s</span>، بينما الهطول و<span dir="ltr" style="unicode-bidi: isolate;">ET</span> بوحدات عمق (ملليمتر) لكل زمن. للمقارنة نحتاج تحويل التصريف الحجمي إلى مكافئ عمق فوق الحوض (مثل <span dir="ltr" style="unicode-bidi: isolate;">mm/year</span>).</p>
</div>

In [ ]:
# Compute total daily discharge based on the mean rate
ds_10years['discharge_total'] = ds_10years.discharge * 60 * 60 * 24

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

# Compute annual discharge
ds_10years['discharge_total'].resample(time = 'YS').sum().to_series().plot.bar()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لا يعرف <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> كيف يضع تسمية للمحور الرأسي لأننا أنشأنا متغيرًا جديدًا دون بيانات وصفية. لكن الوحدات الآن <span dir="ltr" style="unicode-bidi: isolate;">m³/year</span> بعد التحويل، لذا من الأفضل حفظها كسمات.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب مساحة الحوض</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الخطوة الأخيرة هنا هي حساب مساحة الحوض. سنستخدم المساحة لتحويل حجم التصريف إلى ارتفاع ماء مكافئ (ملليمتر) فوق الحوض.</p>
</div>

In [ ]:
basin_albers.crs

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا التحويل من متر مربع إلى كيلومتر مربع بسهولة.</p>
</div>

In [ ]:
basin_area = basin_albers.area.values

# Convert from [square meters] to [square kilometers]
basin_area / 1e6

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كيف تقارن هذه المساحة بقيمة مساحة الصرف المذكورة في <span dir="ltr" style="unicode-bidi: isolate;">HYSETS</span>؟ إنها قريبة جدًا، وهذا مطمئن.</p>
</div>

In [ ]:
ds_10years.drainage_area.values

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب الجريان على مستوى الحوض</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نحوّل حجم التصريف إلى ارتفاع بتقسيمه على مساحة الحوض. النتيجة تقدير لعمق الجريان لكل سنة داخل الحوض.</p>
</div>

In [ ]:
# Get the [m year-1] that this basin drained through Yellowstone River,
#  then convert from [m year-1] to [mm year-1]
runoff_meters_per_yr = ds_10years['discharge_total'].resample(time = 'YS').sum() / basin_area
runoff_mm_per_yr = runoff_meters_per_yr * 1000

runoff_series = runoff_mm_per_yr.values
runoff_series

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">دمج كل شيء</h2>
</div>

In [ ]:
ds_precip = xr.open_dataset('processed/IMERG-Final_precip_monthly_2014-2023.nc')

precip_series = ds_precip.precip_monthly.mean(['lon','lat']).groupby('time.year').sum().values
precip_series

In [ ]:
et_series

In [ ]:
runoff_series

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قبل حساب <span dir="ltr" style="unicode-bidi: isolate;">ΔS</span>، لنقارن التدفقات الثلاثة (<span dir="ltr" style="unicode-bidi: isolate;">P</span> و<span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">R</span>) عبر الزمن. وكما نتوقع، فهي مترابطة بدرجة متوسطة: انخفاض الهطول غالبًا يترافق مع انخفاض الجريان وانخفاض <span dir="ltr" style="unicode-bidi: isolate;">ET</span>.</p>
</div>

In [ ]:
years = np.arange(2014, 2024)
pyplot.plot(years, precip_series, 'b', label = 'Precipitation')
pyplot.plot(years, et_series, 'g', label = 'ET')
pyplot.plot(years, runoff_series, 'r', label = 'Runoff')
pyplot.ylabel('Water Flux (mm per year)')
pyplot.legend()
pyplot.show()

In [ ]:
delta_storage = precip_series - (et_series + runoff_mm_per_yr.values)
delta_storage

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تعطينا النتيجة النهائية</strong> تقديرًا لكيفية تأثر تخزين المياه الجوفية بتوازن المدخلات والمخرجات على مدى هذه السنوات العشر.</p>
</div>

In [ ]:
pyplot.bar(years, height = delta_storage)
pyplot.ylabel('Change in Storage (mm per year)')
pyplot.show()